In [81]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("JunjieChuA3PartA")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

In [82]:
lines1 = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.pl-en.en")
lines2 = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.pl-en.pl")
print(lines1.count())
print(lines2.count())
print(lines1.getNumPartitions())
print(lines2.getNumPartitions())

632565
632565
2
2


In [83]:
def preprocess(lines):
    lowlines=lines.map(lambda line: line.lower())
    words=lowlines.map(lambda word: word.split(' '))
    return(words)
    

In [84]:
words1 = preprocess(lines1)
words2 = preprocess(lines2)
print('En 10:')
print(words1.take(10))
print('Pl 10:')
print(words2.take(10))
print(f'lines number of en: {words1.count()}\n')
print(f'lines number of pl: {words2.count()}\n')

En 10:
[['action', 'taken', 'on', "parliament's", 'resolutions:', 'see', 'minutes'], ['documents', 'received:', 'see', 'minutes'], ['written', 'statements', '(rule', '116):', 'see', 'minutes'], ['texts', 'of', 'agreements', 'forwarded', 'by', 'the', 'council:', 'see', 'minutes'], ['membership', 'of', 'parliament:', 'see', 'minutes'], ['membership', 'of', 'committees', 'and', 'delegations:', 'see', 'minutes'], ['future', 'action', 'in', 'the', 'field', 'of', 'patents', '(motions', 'for', 'resolutions', 'tabled):', 'see', 'minutes'], ['agenda', 'for', 'next', 'sitting:', 'see', 'minutes'], ['closure', 'of', 'sitting'], ['(the', 'sitting', 'was', 'closed', 'at', '11.55', 'p.m.)']]
Pl 10:
[['działania', 'podjęte', 'w', 'wyniku', 'rezolucji', 'parlamentu:', 'patrz', 'protokól'], ['składanie', 'dokumentów:', 'patrz', 'protokół'], ['oświadczenia', 'pisemne', '(art.', '116', 'regulaminu):', 'patrz', 'protokół'], ['teksty', 'porozumień', 'przekazane', 'przez', 'radę:', 'patrz', 'protokół'], ['s

In [85]:
# release the cores for another application!
spark_context.stop()